In [1]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense 

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

2023-01-08 08:20:35.075896: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
symbol = "BTCUSDT"
PERIOD = "15m"

In [3]:
!jupyter nbconvert --to python ../collect/get_newest_price.ipynb
!python ../collect/get_newest_price.py

[NbConvertApp] Converting notebook ../collect/get_newest_price.ipynb to python
[NbConvertApp] Writing 1547 bytes to ../collect/get_newest_price.py


# Origin data

In [4]:
pd_df = pd.read_csv(f"../../datastore/price/{symbol}_{PERIOD}.csv")
pd_df = pd_df.iloc[-1800:]
pd_df

,Kline open time,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
170614,1671521400000,16804.20,16806.96,16780.08,16782.87,1609.84341,1671522299999,2.703421e+07,49421,748.44021,1.256898e+07
170615,1671522300000,16782.16,16822.58,16781.55,16802.83,1737.12871,1671523199999,2.918779e+07,47659,922.81107,1.550524e+07
170616,1671523200000,16802.23,16823.98,16787.71,16794.23,2383.58536,1671524099999,4.005120e+07,64565,1158.76732,1.947117e+07
170617,1671524100000,16793.58,16798.05,16769.25,16785.55,2304.46752,1671524999999,3.867098e+07,57320,1067.86109,1.791966e+07
170618,1671525000000,16784.57,16811.69,16774.82,16808.81,1562.71492,1671525899999,2.624144e+07,48883,809.30295,1.359029e+07
...,...,...,...,...,...,...,...,...,...,...,...
172409,1673136900000,16945.39,16947.59,16937.76,16938.47,1009.82172,1673137799999,1.711011e+07,30794,452.11469,7.660644e+06
172410,1673137800000,16938.47,16941.82,16911.00,16922.55,2607.87832,1673138699999,4.414044e+07,64905,1158.13944,1.960265e+07
172411,1673138700000,16922.55,16928.89,16913.06,16926.66,1260.59103,1673139599999,2.133345e+07,37885,633.09906,1.071425e+07
172412,1673139600000,16926.66,16936.00,16915.25,16933.16,1124.02112,1673140499999,1.902703e+07,34137,570.26887,9.653670e+06


### Format open price and time

In [5]:
pd_df["Open price"] = pd_df["Open price"].apply(lambda price: float(price))
pd_df = pd_df.set_index("Kline open time").sort_index() 
pd_df.index = pd.to_datetime(pd_df.index, unit='ms') + pd.Timedelta('07:00:00')
pd_df

,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
Kline open time,,,,,,,,,,
2022-12-20 14:30:00,16804.20,16806.96,16780.08,16782.87,1609.84341,1671522299999,2.703421e+07,49421,748.44021,1.256898e+07
2022-12-20 14:45:00,16782.16,16822.58,16781.55,16802.83,1737.12871,1671523199999,2.918779e+07,47659,922.81107,1.550524e+07
2022-12-20 15:00:00,16802.23,16823.98,16787.71,16794.23,2383.58536,1671524099999,4.005120e+07,64565,1158.76732,1.947117e+07
2022-12-20 15:15:00,16793.58,16798.05,16769.25,16785.55,2304.46752,1671524999999,3.867098e+07,57320,1067.86109,1.791966e+07
2022-12-20 15:30:00,16784.57,16811.69,16774.82,16808.81,1562.71492,1671525899999,2.624144e+07,48883,809.30295,1.359029e+07
...,...,...,...,...,...,...,...,...,...,...
2023-01-08 07:15:00,16945.39,16947.59,16937.76,16938.47,1009.82172,1673137799999,1.711011e+07,30794,452.11469,7.660644e+06
2023-01-08 07:30:00,16938.47,16941.82,16911.00,16922.55,2607.87832,1673138699999,4.414044e+07,64905,1158.13944,1.960265e+07
2023-01-08 07:45:00,16922.55,16928.89,16913.06,16926.66,1260.59103,1673139599999,2.133345e+07,37885,633.09906,1.071425e+07


In [6]:
fig = go.Figure(data=[go.Candlestick(x=pd_df.index,
                open=pd_df['Open price'],
                high=pd_df['High price'],
                low=pd_df['Low price'],
                close=pd_df['Close price'])])
# set new height and width
fig.update_layout(
    height=800,
    width=1000,
    title_text="BTC/USDT price",
    yaxis_title="Price (BTC/USDT)",
    xaxis_title="Date",
    xaxis_rangeslider_visible=True
)

fig.show()

# Prepare dataset

### Price column

In [7]:
dataset = pd_df.filter(["Open price"]).values
dataset.shape

(1800, 1)

### Splitting ratio

In [8]:
TRAIN_DATA_LENGTH = int(len(dataset) * 0.8)

### Scale data to [0, 1]

In [9]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)
scaled_data

array([[0.6875628 ],
       [0.64870868],
       [0.68408991],
       ...,
       [0.89620097],
       [0.90344645],
       [0.91538123]])

### Split training set

In [10]:
train_data = scaled_data[:TRAIN_DATA_LENGTH]

X_train = []
Y_train = []
for i in range(60,len(train_data)):
    X_train.append(train_data[i-60:i,:])
    Y_train.append(train_data[i,:])

X_train, Y_train = np.array(X_train), np.array(Y_train)
print(X_train.shape, Y_train.shape)

(1380, 60, 1) (1380, 1)


### Split test set

In [11]:
test_data = scaled_data[TRAIN_DATA_LENGTH-60:]

X_test = []
Y_test = []
for i in range(60,len(test_data)):
    X_test.append(test_data[i-60:i,:])
    Y_test.append(test_data[i,:])

X_test, Y_test = np.array(X_test), np.array(Y_test)
print(X_test.shape, Y_test.shape)

(360, 60, 1) (360, 1)


# Training

## Long-short term memory model

In [12]:
model = Sequential()
model.add(LSTM(200, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(200, return_sequences=False))
model.add(Dense(100))
model.add(Dense(X_train.shape[2]))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, Y_train, batch_size=128, epochs=10)

2023-01-08 08:20:46.679548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
11/11 [==============================] - 5s 193ms/step - loss: 0.0519
Epoch 2/10
11/11 [==============================] - 2s 194ms/step - loss: 0.0076
Epoch 3/10
11/11 [==============================] - 2s 196ms/step - loss: 0.0044
Epoch 4/10
11/11 [==============================] - 2s 195ms/step - loss: 0.0028
Epoch 5/10
11/11 [==============================] - 2s 212ms/step - loss: 0.0024
Epoch 6/10
11/11 [==============================] - 2s 196ms/step - loss: 0.0022
Epoch 7/10
11/11 [==============================] - 2s 202ms/step - loss: 0.0021
Epoch 8/10
11/11 [==============================] - 2s 202ms/step - loss: 0.0021
Epoch 9/10
11/11 [==============================] - 2s 217ms/step - loss: 0.0020
Epoch 10/10
11/11 [==============================] - 2s 211ms/step - loss: 0.0019


### Predict and transform [0, 1] to the original scale

In [13]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
predictions[:5]

12/12 [==============================] - 1s 25ms/step


array([[16863.668],
       [16864.611],
       [16866.16 ],
       [16867.402],
       [16869.367]], dtype=float32)

### Visualize the prediction

In [14]:
data = pd_df.filter(["Open price"])
data.shape

(1800, 1)

In [15]:
train = data[:TRAIN_DATA_LENGTH]
valid = data[TRAIN_DATA_LENGTH:].reset_index()
valid["Predict"] = predictions[:, 0]
valid = valid.set_index("Kline open time")
valid["Predict"]= valid["Predict"].apply(lambda price: float(price))

lstm_df = pd.concat([train, valid], axis=0)

In [16]:
fig = px.line(lstm_df[["Open price","Predict"]].dropna(), title="BTC/USDT price" , width=1000, height=800)
fig.show()

## Autoregressive integrated moving average model

In [17]:
from statsmodels.tsa.arima.model import ARIMA

### Scale data to [0, 1] and split train-test set

In [18]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
train, test = scaled_data[:TRAIN_DATA_LENGTH], scaled_data[TRAIN_DATA_LENGTH:]
test.shape

(360, 1)

### Train model

In [19]:
history = train.copy().tolist()

predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit()
    output = model_fit.forecast()
    pred_price = output[0]
    predictions.append(pred_price)
    actual_price = test[t]
    history.append(actual_price)

### Visualize the prediction

In [20]:
train_df, test_df = data[:TRAIN_DATA_LENGTH], data[TRAIN_DATA_LENGTH:]
test_df["Predict"] = scaler.inverse_transform(np.array(predictions).reshape(-1,1)).reshape(-1)
arima_df = pd.concat([train_df, test_df], axis=0)

fig = px.line(arima_df, title="BTC/USDT price" , width=1000, height=800)
fig.show()               

/var/folders/p7/08wxvf195r35gt2pw893ynsw0000gn/T/ipykernel_10173/1968658043.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Model selection with cross validation

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from estimator import LSTMEstimator, SarimaxEstimator, LinearRegressionEstimator

# cross vadiation strategy
tscv = TimeSeriesSplit(n_splits=5)

# mean squared error metric
metric = make_scorer(r2_score)

# models
estimators = [
    SarimaxEstimator(), 
    LinearRegressionEstimator(),
    LSTMEstimator(), 
]

scores_dict = {}

for estimator in estimators:
    scores = cross_val_score(estimator, X_train, Y_train, scoring=metric, cv=tscv, n_jobs=-1)
    print(estimator.__class__.__name__, scores.mean())
    scores_dict[estimator.__class__.__name__] = scores
    
scores_df = pd.DataFrame(scores_dict)
scores_df.index = scores_df.index.map(lambda x: f'fold_{x+1}')
scores_df.loc['mean'] = scores_df.mean()
scores_df.to_csv('../../datastore/scores.csv')

In [ ]:
scores_df

,SarimaxEstimator,LinearRegressionEstimator,LSTMEstimator
fold_1,0.000667,0.000612,0.003439
fold_2,0.000613,0.000457,0.000848
fold_3,0.001515,0.001503,0.001820
fold_4,0.000861,0.000581,0.002987
fold_5,0.000854,0.000534,0.001352
mean,0.000902,0.000738,0.002089


In [ ]:
# find column with lowest mean score
best_estimator = scores_df.loc['mean'].idxmin()
best_estimator

'LinearRegressionEstimator'

In [ ]:
from estimator import Estimator, LinearRegressionEstimator, SarimaxEstimator, LSTMEstimator 
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline

tscv = TimeSeriesSplit()

metric = make_scorer(r2_score)

pipeline = Pipeline(
    steps=[("reg", Estimator())]
)

parameters = [
    {
        'reg__estimator': [LinearRegressionEstimator()]
    },
    {
        'reg__estimator': [SarimaxEstimator()],
        'reg__estimator__order': [(1, 1, 1), (3, 1, 1)],
        'reg__estimator__seasonal_order': [(1, 1, 1, 12), (3, 1, 1, 12)]
    },
    {
        'reg__estimator': [LSTMEstimator()],
        'reg__estimator__epochs': [1, 5, 10],
        'reg__estimator__batch_size': [4, 16, 32],
        'reg__estimator__neurons': [50, 100, 200]
    },
]

estimator.get_params().keys()
grid_search = GridSearchCV(pipeline, parameters, scoring=metric, cv=tscv, n_jobs=2, verbose=0)
grid_search.fit(X_train, Y_train)

In [33]:
grid_search.best_score_

-0.00117153500358897

# Params selection with grid search

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing

data = pd_df.filter(["Open price"])

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data).reshape(-1)
train, test = scaled_data[:TRAIN_DATA_LENGTH], scaled_data[TRAIN_DATA_LENGTH:]
history = train.copy().tolist()

predictions = list()
for t in range(len(test)):
    model = ExponentialSmoothing(history, seasonal_periods=7)
    model_fit = model.fit()
    output = model_fit.forecast()
    pred_price = output[0]
    predictions.append(pred_price)
    actual_price = test[t]
    history.append(actual_price)

In [ ]:

train_df, test_df = data[:TRAIN_DATA_LENGTH], data[TRAIN_DATA_LENGTH:]
test_df["Predict"] = scaler.inverse_transform(np.array(predictions).reshape(-1,1)).reshape(-1)


# concate train_df and test_df to get the full data set
full_df = pd.concat([train_df, test_df], axis=0)


fig = px.line(full_df, height=800, width=1000)
fig.show()

/var/folders/p7/08wxvf195r35gt2pw893ynsw0000gn/T/ipykernel_2422/704318073.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
model = ExponentialSmoothing(data["Open price"].values, seasonal_periods=7)
model_fit = model.fit()
output = model_fit.forecast()
pred_price = output[0]

In [ ]:
pred_price

16832.28719995541